#### **Package installation and loading** ####
The below packages need to be installed prior to running this vignette. Code for package installation is below. There may be additional dependencies required by these packages you may have to install depending on your R environment.

If needed, install the packages using the code below

In [1]:
# install.packages("tidyverse")
# install.packages("Seurat")
# install.packages("HGNChelper")
# install.packages("openxlsx")
# install.packages("DT")

#### **Load the libraries with the code below.** ####

In [2]:
library(tidyverse)
library(Seurat)
library(HGNChelper)
library(openxlsx)
library(DT)

Warning message:
“package ‘purrr’ was built under R version 4.3.1”
Warning message:
“package ‘lubridate’ was built under R version 4.3.1”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


ERROR: Error in library(Seurat): there is no package called ‘Seurat’


#### **Copying BeeNet/BeeNetPLUS output files from Google Cloud** ####
The code below loads your R object from the Google Cloud Platform Bucket. You will need to replace code where outlined below:m

In [ ]:
system("mkdir -p data")

#Replace the URL with the link to the R object output by BeeNetPLUS or your own downstream R object. 
#Navigate to your Google Cloud output directory and locate the .tgz object or R object and copy the path into the code below:
system("gsutil -m cp gs://path-to-data ./data")

#Unzip the file
system("tar -zxf data/*tgz -C ./data")

#### **Load data** ####
##### **HIVE scRNAseq data** #####
Load your R object into the environment. For BeeNetPLUSv1.0 users, change the name of the object using the code below.

The R object below called umi_seurat contains data from HIVE scRNAseq data from blood.

In [ ]:
#Replace the file path for the R object and load in the R object
load("./data/Name_of_File.Rdata")

In [ ]:
# Add a platform metadata column
umi_seurat$platform <- "HIVE"

# Changing name of object
HIVE <- umi_seurat
remove(umi_seurat)

To generate an R object from count matrices from BeeNet, see our Seurat Analysis Tutorial.

#### **10X data** ####
Load matching 10X data and create an R object. The count matrix for 10x data is typically in a folder named filtered_featured_bc_matrix. This code below reads in the data and creates one Seurat object containing the 10x data.



In [ ]:
# list the filtered_featured_bc_matrix folders for each sample
count_folder_10x <- "path_to_10x_featured_filtered_bc_matrix_folder/"

In [ ]:
# Create Seurat objects for each sample
seurat_10x <- CreateSeuratObject(counts = Read10X(count_folder_10x))

# Add platform information
seurat_10x$platform <- "10x"

#### **Merge data** ####
Without any integration, you can merge the two Seurat objects to create one object. However, the differences in the two platforms platforms cause cells to cluster by platform instead of by cell type. Integration will allow you to analyze the data for biological, rather than technical, variation.



In [ ]:
# merging 2 datasets without integration
obj <- merge(HIVE, seurat_10x)

# Cluster new object
obj <- NormalizeData(obj)
obj <- FindVariableFeatures(obj, selection.method = "vst", nfeatures = 2000)
obj <- ScaleData(obj)
obj <- RunPCA(obj, verbose = FALSE)
obj <- RunUMAP(obj, dims=1:30,verbose = FALSE)
obj <- FindNeighbors(obj,verbose = FALSE)
obj <- FindClusters(obj, verbose = FALSE)
obj <- BuildClusterTree(obj,reorder=TRUE,reorder.numeric=TRUE)
obj$seurat_clusters <- obj$tree.ident

# Plot results
DimPlot(obj, group.by = "platform", cols = c("grey", "orange"))

#### **Integrating data** ####
The two data sets can be integrated using Seurat functions. First, identify features for integration using SelectIntegrationFeatures(). Next, find integration anchors using FindIntegrationAnchors(). Finally, integrate the data using IntegrateData(). Integration creates a new assay called “integrated”. This assay automatically becomes the default assay in the object and contains variable features that have been integrated. The full data sets for each sample remain in the object in the RNA assay.

The resulting data shows that data from both platforms can integrate together and clustering occurs by cell type instead of platform.

In [ ]:
# remove old object 
remove(obj)

# Normalize and identify variable features in each data set
HIVE <- NormalizeData(HIVE)
HIVE <- FindVariableFeatures(HIVE, selection.method = "vst", nfeatures = 2000)
HIVE <- ScaleData(HIVE)

seurat_10x <- NormalizeData(seurat_10x)
seurat_10x <- FindVariableFeatures(seurat_10x, selection.method = "vst", nfeatures = 2000)
seurat_10x <- ScaleData(seurat_10x)

# Add percent.mito reads to 10x data - this metadata already exists in BeeNetPlus generated R objects
seurat_10x[["percent.mito"]] <- PercentageFeatureSet(seurat_10x, pattern = "^MT-") / 100

# Find features for integration
features <- SelectIntegrationFeatures(object.list = c(HIVE, seurat_10x))

# Find anchors for integration using the RNA assay for each object
anchors <- FindIntegrationAnchors(object.list = c(HIVE, seurat_10x), anchor.features = features, 
                                  assay = c("RNA", "RNA"))

# Integrate data
obj <- IntegrateData(anchorset = anchors)

You can easily change the default assay with the code below.

In [ ]:
# Cluster new data
DefaultAssay(obj) <- "integrated"

obj

In [ ]:
## An object of class Seurat 
## 81807 features across 4361 samples within 3 assays 
## Active assay: integrated (2000 features, 2000 variable features)
##  2 other assays present: RNA, SCT

Following integration, the new assay “integration” needs to be reclustered.

In [ ]:
# Run the standard workflow for visualization and clustering
obj <- ScaleData(obj, verbose = FALSE)
obj <- RunPCA(obj, npcs = 30, verbose = FALSE)
obj <- RunUMAP(obj, reduction = "pca", dims = 1:30)
obj <- FindNeighbors(obj, reduction = "pca", dims = 1:30)
obj <- FindClusters(obj, resolution = 0.3, verbose = F) 
obj <- BuildClusterTree(obj,reorder=TRUE,reorder.numeric=TRUE,
                        verbose = F)
obj$seurat_clusters <- obj$tree.ident

# Visualize results
DimPlot(obj, group.by = "platform", cols = c("grey", "orange"))

#### **Annotating cell types** ####
Below, we use scType to automatically annotate cell types by cluster.



In [ ]:
# Using scType to annotate cell types

## Download the reference scripts and mar
source("https://raw.githubusercontent.com/IanevskiAleksandr/sc-type/master/R/gene_sets_prepare.R")
source("https://raw.githubusercontent.com/IanevskiAleksandr/sc-type/master/R/sctype_score_.R")
source("https://raw.githubusercontent.com/IanevskiAleksandr/sc-type/master/R/auto_detect_tissue_type.R")

# Download reference data for different tissue types
db_ = "https://raw.githubusercontent.com/IanevskiAleksandr/sc-type/master/ScTypeDB_full.xlsx"

# Define the tissue. These are blood samples so Immune system is the right choice
tissue<- "Immune system"

# prepare gene sets
gs_list = gene_sets_prepare(db_, tissue)

es.max = sctype_score(scRNAseqData = obj[["integrated"]]@scale.data, 
                      scaled = TRUE, 
                      gs = gs_list$gs_positive, 
                      gs2 = gs_list$gs_negative) 

# merge by cluster
cL_resutls = do.call("rbind", lapply(unique(obj@meta.data$seurat_clusters), function(cl){
    es.max.cl = sort(rowSums(es.max[ ,rownames(obj@meta.data[obj@meta.data$seurat_clusters==cl, ])]), decreasing = !0)
    head(data.frame(cluster = cl, type = names(es.max.cl), scores = es.max.cl, ncells = sum(obj@meta.data$seurat_clusters==cl)), 10)
}))
sctype_scores = cL_resutls %>% group_by(cluster) %>% top_n(n = 1, wt = scores)  

# set low-confident (low ScType score) clusters to "unknown"
sctype_scores$type[as.numeric(as.character(sctype_scores$scores)) < sctype_scores$ncells/4] = "Unknown"
#print(sctype_scores[,1:3])

obj_cluster_id <- sctype_scores

# save old idents for reference
obj[["old.ident"]] <- Idents(object = obj)


obj@meta.data$customclassif = ""
        for(j in unique(sctype_scores$cluster)){
            cl_type = sctype_scores[sctype_scores$cluster==j,]; 
            obj@meta.data$customclassif[obj@meta.data$seurat_clusters == j] = as.character(cl_type$type[1])
        }
        
obj[["cell_type"]]<- obj$customclassif

# Visualize cell types
DimPlot(obj, group.by = "cell_type") 


#### **Feature Plots** ####
Use the code below to look at feature plots. You can plot QC metrics from the metadata or marker genes of interest. BeeNetPLUSv1.0 users will have QC meta data in their Seurat objects. Some of this metadata is not present in the 10X count matrices. These variables will only plot in HIVE data.



In [ ]:
# Look at marker genes and QC metrics
FeaturePlot(obj, c("IL7R", "nCount_RNA", "nFeature_RNA", "percent.mito"))

Find marker genes
You can find marker genes between clusters or cell types using the integrated data.

In [ ]:
# Set cell idents to clusters
Idents(obj) <- obj$seurat_clusters

# Find marker genes between clusters 1 and 2
markers_clust <- FindMarkers(obj, ident.1 = 1, ident.2 = 2, min.pct = 0.25)

# Display top marker genes
datatable(markers_clust[1:5,])

In [ ]:
# Set cell idents to cell type
Idents(obj) <- obj$cell_type

# Find marker genes
markers_ct <- FindAllMarkers(obj, min.pct = 0.25)

# Display top marker gene for each cell type
markers_ct %>% group_by(cluster) %>%
        slice(1) %>% datatable()

In [ ]:
# Find marker genes using the data in the RNA assay
RNA_markers <- FindAllMarkers(obj, assay = "RNA", min.pct = 0.25)

# Display top marker
RNA_markers %>% group_by(cluster) %>%
        slice(1) %>% datatable()

Visualize marker genes by platform

In [ ]:
plot_marks <- markers_ct %>% 
        group_by(cluster) %>%
        slice(1)

DotPlot(obj, plot_marks$gene, assay = "integrated", split.by = "platform", 
        cols = c("orange", "blue"), dot.scale = 8) + 
        RotatedAxis() + 
        theme(text = element_text(size = 8),
              axis.text.y = element_text(size= 8))


 Integration with SCTransform
You can also integrate using SCTransform to regress out variables like percent mitochondrial reads.

In [ ]:
# Use SCTransform to regress out variation in percent.mito and normalize data
HIVE <- SCTransform(HIVE, vars.to.regress = "percent.mito", verbose = FALSE)
seurat_10x <- SCTransform(seurat_10x, vars.to.regress = "percent.mito", verbose = FALSE)

# Find markers for integration 
features <- SelectIntegrationFeatures(object.list = c(HIVE, seurat_10x), nfeatures = 3000)
obj_list <- PrepSCTIntegration(object.list = c(HIVE, seurat_10x), anchor.features = features)

# Find anchors and integrate data using SCT normalization method
SCTanchors <- FindIntegrationAnchors(object.list = obj_list, normalization.method = "SCT",
                                         anchor.features = features)
obj_SCT <- IntegrateData(anchorset = SCTanchors, normalization.method = "SCT")

# Cluster and visualize
obj_SCT <- ScaleData(obj_SCT, verbose = FALSE)
obj_SCT <- RunPCA(obj_SCT, npcs = 30, verbose = FALSE)
obj_SCT <- RunUMAP(obj_SCT, reduction = "pca", dims = 1:30)
obj_SCT <- FindNeighbors(obj_SCT, reduction = "pca", dims = 1:30)
obj_SCT <- FindClusters(obj_SCT, resolution = 0.5, verbose = F)
obj_SCT <- BuildClusterTree(obj_SCT,reorder=TRUE,reorder.numeric=TRUE,
                        verbose = F)
obj_SCT$seurat_clusters <- obj_SCT$tree.ident

# Visualize results
DimPlot(obj_SCT, group.by = "platform", cols = c("grey", "orange"))